In [20]:
import numpy, matplotlib
directory = '../data/'
filePaths = ['a_example.txt', 'b_read_on.txt', 'c_incunabula.txt','d_tough_choices.txt', 'e_so_many_books.txt', 'f_libraries_of_the_world.txt']

In [22]:
def process_file(filePath):
    with open(directory+filePath,'r') as file:
        content = file.read().split('\n')[:-1];
        B,L,D = content[0].split()
        B,L,D = int(B),int(L),int(D)
    bookValues = tuple(content[1].split())
    libraries = []
    for i in range(L):
        N,T,M = content[2+2*i].split()
        bookIds = tuple(content[2+2*i+1].split())
        libraries.append(tuple(((N,T,M), bookIds)))
for filePath in filePaths:
    process_file(filePath)

6 2 7
100000 100 1000
100000 10000 100000
78600 30000 30001
100000 1000 200
100000 1000 700
